# Week 7: Multilayer Perceptron Neural Networks

### Aims:
* To gain some experience in constructing multilayer perceptron networks and solving problems
with them.
* To produce some assessable work for this subject.

### Procedure:
Some of the questions in this Prac use python libraries, in particular the PyTorch library
(https://pytorch.org/) developed by Meta AI1. If you are not familiar with python it should still be
possible to work through the prac questions using the examples/partial code provided.

### Question 1: The XOR dataset is a classic toy problem in the history of neural networks. Use the
code provided to try and train a 2-2-1 MLP to learn XOR. Use the code to verify for yourself the
following key points:
The result of the training is sensitive to the initialization of the weights.
The result of the training is sensitive to the learning rate.
The result of the training is sensitive to the number of epochs/weight updates.
The result of the training is sensitive to the scale of the (input) data.
Hint: think about what variable(s) you need to look at to verify these things. Add some lines to
print out and/or plot some of the variables before/during/after training.


Introduction to PyTorch
Question 2: On the course website you will find a link to a google colab notebook. Read and step
through this notebook to see how to create and train MLPs in PyTorch.
MNIST Data
Question 3: Train an MLP on the MNIST data (available on the course website). You will need to
make several “design choices” to do this. Make sure you split your dataset into a training,
validation and test set. Your answer for this question should describe the design choices you made
to create and train your network and a report of your results. You can also include error graphs or
any other output you feel is useful.
Python help: some example code is provided using either sklearn or pytorch.
Matlab help: run the Neural Network Pattern Recognition tool from the command line
(nprtool). You can import a dataset (e.g. cancer) and step through the creation, training and
evaluation of a neural network inside the tool. You can also generate code from this
1 Note that this is originally based on Torch, developed in Lua and first released in 2002
(https://en.wikipedia.org/wiki/Torch_(machine_learning)).
process. You could then modify this code to apply to the MNIST data (or go the opposite
way: import the MNIST data into the GUI tool).
There are other apps that could be used: Classification Learner or Deep Network Designer.
The following code may be used in formatting the data. You will have to change it slightly
to use the test and train dataset.
load('mnist_train.mat');
labels = [];
for i = 1:10
labels = [labels, train_labels == i];
end
Question 4: The weight matrix and bias vector in first hidden layer apply a linear transformation
before the activation function is applied. The ith row of the weight vector contains the weightings
of the 728 inputs to the ith neuron. The importance of each input on the ith neuron can then be
visualised by drawing the ith row of the weight vector as a greyscale image. Visualise 9 rows of the
weight matrix.
Python help: see the example code provided.
Matlab help:
• One way to get the weight matrix of the trained network is to click “Matlab Matrix
Only Function” in the “Deploy Solution” dialogue of nprtool. You will need to copy
and paste IW1_1 and x1_step1_keep into a new script.
• Each row will have to be normalised so that every element is in [0,1].
• reshape() and imshow() may be useful.
Extra Topics to Note/Explore:
• Numerical methods (like gradient descent) typically require termination conditions. This
could be a simple as training for a fixed number of epochs, however other things could also
be considered (e.g. stopping if the magnitude of the gradient drops below some small value,
or if the loss has not decreased by more than some small value in the last few steps. These
choices are effectively hyperparameters of the configuration of any training experiment.
Software libraries will typically have some default settings.
• You might like to explore using different training algorithms beyond stochastic gradient
descent (e.g. Adam).

"Hello World"